In [6]:
%%time
#coding=utf-8

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
import os
import imp
import signal
import traceback
import sys
import json


SCRIPT_DIR = os.path.dirname(os.path.realpath('__file__'))


def signal_handler(signum, frame):
    raise Exception("Timed out!")


class Checker(object):
    def __init__(self):
        # ВНИМАНИЕ !!!
        # При тестировании seed будет изменён
        # Не переобучитесь!
        random_gen = np.random.RandomState(42)
        weights = (0.05 + random_gen.exponential(0.75, size=15)) * 2
        X_data = random_gen.uniform(0., 4, size=(40, 15))
        errors = random_gen.normal(0., 2., size=40)
        split_pos = 25
        self.X_train = X_data[:split_pos]
        self.errors_train = errors[:split_pos]
        self.X_test = X_data[split_pos:]
        self.errors_test = errors[split_pos:]
        self.weights = weights

        self.applications = 0

    def check(self, script_path):
        try:
            signal.signal(signal.SIGALRM, signal_handler)
            signal.alarm(240)
            algo_impl = imp.load_source('algo_impl_{}'.format(self.applications), script_path)
            self.applications += 1
            algo = algo_impl.Optimizer()
            algo.fit(np.array(self.X_train), np.dot(self.X_train, self.weights) + self.errors_train)
            
            saved_moneys = 0.
            first = 0
            second = 0
            for budget, target_error in zip(self.X_test, self.errors_test):
                origin_budget = np.array(budget)
                optimized_budget = np.array(algo.optimize(origin_budget))

                if ((origin_budget * 0.95 <= optimized_budget) & (optimized_budget <= origin_budget * 1.05)).all():
                    if np.dot(optimized_budget, self.weights) >=  np.dot(origin_budget, self.weights):
                        saved_moneys += np.sum(origin_budget) - np.sum(optimized_budget)
                        first += np.sum(optimized_budget)
                        second += np.sum(origin_budget)
            print(first / second)
            return saved_moneys
        except:
            traceback.print_exception(*sys.exc_info())
            return None


if __name__ == '__main__':
    print(Checker().check(SCRIPT_DIR + '/ad_budget_dankin.py'))



[ 3.96202057  1.65047071  1.48807234  3.10565184  1.36321416  3.7230293
  3.43365101  1.71597611  3.00348427  3.0181715   0.41249548  3.61021163
  2.02100949  3.30582986  1.2801984 ]
None
CPU times: user 10.8 ms, sys: 4.19 ms, total: 15 ms
Wall time: 16.3 ms


Traceback (most recent call last):
  File "<timed exec>", line 54, in check
  File "/Users/nickdanilov/Desktop/git_projects/dmia/Data_Mining_in_Action_2018_Spring/industry/hw03/ad_budget_dankin.py", line 23, in optimize
    res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
  File "/Users/nickdanilov/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog.py", line 993, in linprog
    bounds=bounds, callback=callback, **options)
  File "/Users/nickdanilov/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog.py", line 673, in _linprog_simplex
    n_artificial = meq + np.count_nonzero(bub < 0)
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
